In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
import json, datetime, pandas
pandas.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import ipywidgets as widgets 
from IPython.display import display, clear_output, Markdown

In [10]:
class LOB:
    # variables can be access externally
    DATA={}
    TRADES = {}
    FILTERED_DATA = {}
    def __init__(self, symbols, csv_file, csv_trades=None):
        self.symbols = symbols
        self.DATA = pandas.read_csv(csv_file,sep=';')
        if csv_trades is not None:
            self.TRADES = pandas.read_csv(csv_trades,sep=';')
            self.DATA = pandas.merge(left=self.DATA, right=self.TRADES[['time','price', 'qty']], how='inner', left_on='timestamp', right_on='time')
            self.DATA .drop(["time"], axis = 1, inplace = True)
        self.DATA.set_index('timestamp', inplace=True)
    def lob_stats_analyser(self):
        outt = widgets.Output()
        dropdown_symbol = widgets.Dropdown(options=self.symbols, description='Symbols:')
        stats_checkbox = [widgets.Checkbox(description=stats, value=True,indent=False, disabled=False) for stats in self.DATA.columns]
        stats_checkboxes = widgets.HBox(stats_checkbox)
        rolling = widgets.FloatSlider(
                 value=50,
                 min=1,
                 max=200,
                 step=1,
                 disabled=False,
                 description='Rolling:',
                )
        timerange = widgets.IntRangeSlider(
                        value=[0,0],
                        min=0,
                        max=0,
                        step=5,
                        description='Range:',
                        disabled=False,
                        continuous_update=False,
                        orientation='horizontal',
                        readout=True,
                        readout_format='d',
                        layout=dict(width='95%')
                        )
        timerange.max = self.DATA.index.max()
        timerange.min = self.DATA.index.min()
        timerange.value = [self.DATA.index.min(), self.DATA.index.max()]

        def plot_time_range(start, end, normalized):
            with outt:
                clear_output()
                timerange.value = [start, end]
                print("Start time:", datetime.datetime.fromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S'),
                        "  End time:", datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S'))
                selected_stats = [checkbox.description for checkbox in stats_checkbox if checkbox.value]
                if normalized:
                    self.FILTERED_DATA = self.DATA[(self.DATA.symbol == dropdown_symbol.value) & (self.DATA.index > start) & 
                        (self.DATA.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
                    self.FILTERED_DATA = self.FILTERED_DATA.apply(lambda x: (x-x.min())/(x.max()-x.min()))
                else:
                    self.FILTERED_DATA = self.DATA[(self.DATA.symbol == dropdown_symbol.value) & (self.DATA.index > start) & 
                        (self.DATA.index < end)][selected_stats].rolling(int(rolling.value)).mean().dropna()
                plt.rcParams['figure.figsize'] = [15,7]
                plt.rcParams['timezone'] = 'America/Sao_Paulo'

                fig,ax = plt.subplots()
                x = mdate.epoch2num(self.FILTERED_DATA.index)
                fmt = mdate.DateFormatter('%H:%M:%S')

                ax.xaxis.set_major_formatter(fmt)
                plt.xticks( rotation=25 )
                plt.plot_date(x, self.FILTERED_DATA, linestyle='solid', marker='None')

                plt.legend(self.FILTERED_DATA.columns)
                plt.show()
        def on_btn_filter_range_clicked(b):
            plot_time_range(timerange.value[0], timerange.value[1], False)
        def on_btn_filter_range_norm_clicked(b):
            plot_time_range(timerange.value[0], timerange.value[1], True)
        btn_filter_range = widgets.Button(description='Filter',button_style='danger', disabled=False)
        btn_filter_range.on_click(on_btn_filter_range_clicked)
        btn_filter_range_norm = widgets.Button(description='Filter w/ Norm',button_style='warning', disabled=False)
        btn_filter_range_norm.on_click(on_btn_filter_range_norm_clicked)
        btns = widgets.HBox([dropdown_symbol,btn_filter_range, btn_filter_range_norm])

        return widgets.VBox([stats_checkboxes, rolling, timerange,btns, outt])

In [13]:
#symbols = ["vale3","petr4", "bpac11", "ceab3", "abev3", "b3sa3", "embr3", "ggbr4", "tots3"]
symbols = ["vale3"]
file = '20210420_vale3_aba.log.csv'
trades = '20210420_vale3_bbt.log.csv'
lob = LOB(symbols, file, trades)
lob.lob_stats_analyser()

In [ ]:
lob.DATA.head()

In [7]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html
lob.FILTERED_DATA.corr()

,book_imbalance,price
book_imbalance,1.00,-0.72
price,-0.72,1.00


In [8]:
def df_derived_by_shift(df,lag=0,NON_DER=[]):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in NON_DER:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pandas.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=i)
            i+=1
        df = pandas.concat([df, dfn], axis=1, join_axes=[df.index])
    return df
df_new = df_derived_by_shift(lob.FILTERED_DATA, 10).dropna()
CORR = df_new.corr()

/Library/Python/3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.


In [9]:
#CORR['book_imbalance']['book_imbalance']
pandas.set_option('display.max_rows', 1000) 
pandas.set_option('display.max_columns', 1000) 

#CORR.book_imbalance
CORR

,book_imbalance,price,book_imbalance_1,book_imbalance_2,book_imbalance_3,book_imbalance_4,book_imbalance_5,book_imbalance_6,book_imbalance_7,book_imbalance_8,book_imbalance_9,book_imbalance_10,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,price_10
book_imbalance,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,0.99,0.99,0.99,0.99,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
price,-0.72,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
book_imbalance_1,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,0.99,0.99,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_2,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,0.99,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_3,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_4,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_5,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_6,1.00,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_7,0.99,-0.72,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72
book_imbalance_8,0.99,-0.72,0.99,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72,-0.72


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
colormap = plt.cm.RdBu
plt.figure(figsize=(20,15))
#plt.title(u'6 hours', y=1.05, size=16)

mask = np.zeros_like(df_new.corr())
mask[np.triu_indices_from(mask)] = True

svm = sns.heatmap(df_new.corr(), mask=mask, linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)